<a href="https://colab.research.google.com/github/yosshy2020/intro_git/blob/master/cat2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install catboost

In [3]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as optimize
from scipy.stats import skew
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import xgboost as xgb
from catboost import Pool, CatBoostRegressor
%matplotlib inline

train = pd.read_csv('/content/drive/MyDrive/biginners/train.csv')
test = pd.read_csv('/content/drive/MyDrive/biginners/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/biginners/sample_submit.csv')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11900 entries, 0 to 11899
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           11900 non-null  int64 
 1   age             11900 non-null  int64 
 2   workclass       11900 non-null  object
 3   fnlwgt          11900 non-null  int64 
 4   education       11900 non-null  object
 5   education-num   11900 non-null  int64 
 6   marital-status  11900 non-null  object
 7   occupation      11900 non-null  object
 8   relationship    11900 non-null  object
 9   race            11900 non-null  object
 10  sex             11900 non-null  object
 11  native-country  11900 non-null  object
 12  Y               11900 non-null  int64 
dtypes: int64(5), object(8)
memory usage: 1.2+ MB


In [5]:
all_data = pd.concat((train.loc[:,'age':'native-country'],
                      test.loc[:,'age':'native-country']))

In [6]:
# fnlwgtクラス
all_data = all_data.replace({'fnlwgt':[24694]}, 0)
all_data = all_data.replace({'fnlwgt':[132652]}, 1)
all_data = all_data.replace({'fnlwgt':[34572]}, 2)

In [7]:
all_data = all_data.drop([
                          # "workclass",
                          "education-num",
                          # "sex",
                          "race"
                          ], axis=1)

In [8]:
all_data.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'marital-status',
       'occupation', 'relationship', 'sex', 'native-country'],
      dtype='object')

In [9]:
categories = all_data.columns[all_data.dtypes == 'object']
categories

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'sex', 'native-country'],
      dtype='object')

In [10]:
le = LabelEncoder()

for cat in categories:
  le = LabelEncoder()
  print(cat)
  if all_data[cat].dtypes == "object":
    le = le.fit(all_data[cat])
    all_data[cat] = le.transform(all_data[cat])

workclass
education
marital-status
occupation
relationship
sex
native-country


In [11]:
train_X = all_data[:train.shape[0]]
test_X = all_data[train.shape[0]:]
train_Y= train.Y

In [12]:
folds = 5
kf = KFold(n_splits=folds, shuffle=True)

In [13]:
categories = ['fnlwgt', 'education', 'marital-status', 'occupation',
       'relationship', 'native-country']

In [14]:
for col in categories:
  train_X[col] = train_X[col].astype("int8")
  test_X[col]  = test_X[col].astype("int8")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
# params = {
#         'leaf_estimation_method': 'Gradient',
#         'learning_rate': 0.1,
#         'max_depth': 3,
#         'bootstrap_type': 'Bernoulli',
#         'objective': 'MultiClass',
#         'eval_metric': 'MultiClass',
#         'subsample': 0.8,
#         'random_state': 1234,
#         'verbose': 20}

In [16]:
# params = {
#     'boosting_type': 'gbdt',
#     'objective':'regression',
#     'learning_rate': 0.05,
#     #'is_unbalance': 'true',  #because training data is unbalance (replaced with scale_pos_weight)
#     'num_leaves': 31,  # we should let it be smaller than 2^(max_depth)
#     'max_depth': -1,  # -1 means no limit
#     'min_child_samples': 20,  # Minimum number of data need in a child(min_data_in_leaf)
#     'max_bin': 255,  # Number of bucketed bin for feature values
#     'subsample': 0.6,  # Subsample ratio of the training instance.
#     # 'subsample_freq': 0,  # frequence of subsample, <=0 means no enable
#     # 'colsample_bytree': 0.3,  # Subsample ratio of columns when constructing each tree.
#     # 'min_child_weight': 5,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
#     # 'subsample_for_bin': 200000,  # Number of samples for constructing bin
#     # 'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
#     # 'reg_alpha': 0,  # L1 regularization term on weights
#     'reg_lambda': 0,  # L2 regularization term on weights
#     # 'nthread': 8,
#     'verbose': 10,
# }

In [17]:
 params = {
        'iterations' : 125,                         
        'depth' : 4,                                       
        'learning_rate' : 0.2999453264309889,               
        'random_strength' :31,                       
        'bagging_temperature' :29.043339885326265, 
        'od_type': 'Iter', 
        'od_wait' :31,
    }

In [18]:
model = CatBoostRegressor(**params)

In [19]:
models = [ ]
accuracy =[ ]

for train_index, val_index in kf.split(train_X):
  X_train =train_X.iloc[train_index]
  X_valid =train_X.iloc[val_index]
  y_train =train_Y.iloc[train_index]
  y_valid =train_Y.iloc[val_index]

  model.fit(X_train, y_train, 
          early_stopping_rounds=50,
          verbose_eval=10,) 
  
  y_pred = model.predict(X_valid)
  print(accuracy_score(y_valid, np.round(y_pred)))
  accuracy.append(accuracy_score(y_valid, np.round(y_pred)))


  models.append(model)

0:	learn: 0.4205602	total: 49.9ms	remaining: 6.19s
10:	learn: 0.3734851	total: 69.5ms	remaining: 721ms
20:	learn: 0.3595956	total: 88.7ms	remaining: 439ms
30:	learn: 0.3571115	total: 103ms	remaining: 312ms
40:	learn: 0.3468971	total: 125ms	remaining: 256ms
50:	learn: 0.3368840	total: 147ms	remaining: 214ms
60:	learn: 0.3327389	total: 167ms	remaining: 175ms
70:	learn: 0.3302547	total: 185ms	remaining: 141ms
80:	learn: 0.3285447	total: 207ms	remaining: 113ms
90:	learn: 0.3270324	total: 228ms	remaining: 85.2ms
100:	learn: 0.3259075	total: 249ms	remaining: 59.1ms
110:	learn: 0.3246546	total: 270ms	remaining: 34.1ms
120:	learn: 0.3236743	total: 288ms	remaining: 9.53ms
124:	learn: 0.3232461	total: 295ms	remaining: 0us
0.8457983193277311
0:	learn: 0.4231264	total: 1.85ms	remaining: 230ms
10:	learn: 0.3742696	total: 20.7ms	remaining: 215ms
20:	learn: 0.3604212	total: 40.3ms	remaining: 199ms
30:	learn: 0.3577917	total: 62.6ms	remaining: 190ms
40:	learn: 0.3475681	total: 83.6ms	remaining: 171ms


In [20]:
print('accuracy:{}'.format(np.mean(accuracy)))

accuracy:0.8389075630252101
